# **Assignment 2**

## **ECON8502 - Structural Microeconometrics**

### *Conor Bayliss*

#### **Setup**

The first thing we do is run the setup code. This reads in the data and creates a NamedTuple "data_mle".

In [9]:
using CSV, DataFrames, DataFramesMeta, Optim
include("C:\\Users\\bayle\\Documents\\Github\\micro_labour\\Transfers.jl")

data = @chain begin
    CSV.read("C:\\Users\\bayle\\Documents\\Github\\micro_labour\\hw1\\hw1_micro_labour_data\\MainPanelFile.csv", DataFrame, missingstring = "NA")
    #@select :MID :year :wage :hrs :earn :SOI :CPIU :WelfH :FSInd :num_child :age
    @subset :year.>=1985 :year.<=2010 # 1985-2010 subset
    @transform :AFDC = :WelfH.>0 # AFDC dummy
    @rename :FS = :FSInd # FS dummy
    @transform :P = :FS + :AFDC :H = min.(2, round.(Union{Int64, Missing}, :hrs / (52*20))) # Create P as defined in class (0,1,2) and create H as a (0,1,2) indicator on hrs worked (0,20,40)
    @subset .!ismissing.(:P) .&& .!ismissing.(:H) # Drop missing values
    @transform @byrow :wage = begin # Create wage variable
        if :hrs>0 && :earn>0 # If hours worked and earnings are positive
            return :earn / :hrs / :CPIU # Calculate wage
        else
            return missing # Else return missing
        end
    end 
end

Row,Column1,year,MID,intnum,def_flag,age,knum,num_child,age_youngest,age_oldest,num_0_5,num_6_12,y_first,cds_sample,mstat,mpair,relhead,sn,state,fips,SOI,time_limit,year_TL,H.WInc,TotInc,OthInc,WelfH,WelfH_tu,WelfMonths,FS,CPIU,tau,hrs,earn,wage,N,AFDC,P,H
,Int64,Int64,Int64,Int64?,Bool,Int64,Int64,Int64,Float64,Int64,Int64,Int64,Int64,Bool,Bool,Int64?,Int64?,Int64?,String3,Int64,Int64,Int64,Int64,Float64?,Float64?,Float64?,Float64?,Int64?,Int64?,Int64?,Float64,Float64?,Int64?,Float64?,Float64?,Float64?,Bool?,Int64?,Int64?
1,5,1994,4031,5382,false,21,3,2,2.0,4,2,0,1990,true,true,0,10,1,TN,47,43,18,1997,0.0,8157.0,0.0,1704.0,1,12,1,0.860812,missing,0,0.0,missing,6453.0,true,2,0
2,6,1995,4031,2239,false,22,3,2,3.0,5,2,0,1990,true,true,0,10,1,TN,47,43,18,1997,0.0,7500.0,0.0,1704.0,5,12,1,0.88496,missing,0,0.0,missing,5796.0,true,2,0
3,7,1996,4031,4246,false,23,3,3,0.0,6,2,1,1990,true,true,0,10,1,TN,47,43,18,1997,0.0,7512.0,0.0,1704.0,5,12,1,0.910948,51.8056,0,0.0,missing,5808.0,true,2,0
4,9,1998,4031,1702,false,25,3,3,2.0,8,1,2,1990,true,true,0,10,1,TN,47,43,18,1997,0.0,8000.0,0.0,0.0,0,0,1,0.946664,missing,0,0.0,missing,8000.0,false,1,0
5,11,2000,4031,285,false,27,3,3,4.0,10,1,2,1990,true,true,1,20,2,TN,47,43,18,1997,30400.0,37760.0,0.0,0.0,0,0,0,1.0,missing,120,400.0,3.33333,37360.0,false,0,0
6,13,2002,4031,1427,false,29,3,3,6.0,12,0,3,1990,true,true,1,20,2,TN,47,43,18,1997,27326.0,40574.0,0.0,0.0,0,0,0,1.04457,27.5,0,0.0,missing,40574.0,false,0,0
7,15,2004,4031,1157,false,31,3,3,8.0,14,0,2,1990,true,true,1,20,2,TN,47,43,18,1997,27000.0,40536.0,0.0,0.0,0,0,0,1.09708,missing,0,0.0,missing,40536.0,false,0,0
8,17,2006,4031,196,false,33,3,3,10.0,16,0,1,1990,true,true,1,20,2,TN,47,43,18,1997,25000.0,40594.0,0.0,142.0,6,1,1,1.17054,missing,0,0.0,missing,40452.0,true,2,0
9,19,2008,4031,684,false,35,3,3,12.0,18,0,1,1990,true,true,1,20,2,TN,47,43,18,1997,25000.0,43398.0,0.0,0.0,0,0,1,1.25008,missing,0,0.0,missing,43398.0,false,1,0


In [12]:
data_mle = (;P = Int64.(data.P), H = Int64.(data.H), year = data.year, age = data.age,
            soi = data.SOI, num_kids = data.num_child, cpi = data.CPIU,
            logwage = log.(coalesce.(data.wage,1.)),wage_missing = ismissing.(data.wage))

(P = [2, 2, 2, 1, 0, 0, 0, 2, 1, 0  …  2, 2, 1, 2, 0, 0, 1, 1, 1, 1], H = [0, 0, 0, 0, 0, 0, 0, 0, 0, 2  …  0, 0, 2, 2, 2, 2, 0, 0, 0, 0], year = [1994, 1995, 1996, 1998, 2000, 2002, 2004, 2006, 2008, 1990  …  1991, 1992, 1991, 1992, 1991, 1992, 1991, 1992, 1991, 1992], age = [21, 22, 23, 25, 27, 29, 31, 33, 35, 17  …  21, 22, 23, 24, 39, 40, 30, 31, 25, 26], soi = [43, 43, 43, 43, 43, 43, 43, 43, 43, 17  …  44, 44, 7, 7, 5, 5, 39, 39, 39, 39], num_kids = [2, 2, 3, 3, 3, 3, 3, 3, 3, 1  …  2, 2, 2, 2, 3, 3, 4, 4, 2, 2], cpi = [0.860812349005761, 0.884959812302546, 0.910948243820851, 0.946664188812488, 1.0, 1.04457233785542, 1.09707768072849, 1.17054218546739, 1.25008130459022, 0.758792510685746  …  0.790785866939231, 0.8148346032336, 0.790785866939231, 0.8148346032336, 0.790785866939231, 0.8148346032336, 0.790785866939231, 0.8148346032336, 0.790785866939231, 0.8148346032336], logwage = [0.0, 0.0, 0.0, 0.0, 1.2039728043259361, 0.0, 0.0, 0.0, 0.0, 0.8230555833449215  …  0.0, 0.0, 2.651257

##### The Model

In class, we defined $P_j \in \{0,1,2\}$ as the indicator for welfare participation, and above we have created $H_j \in \{0,1,2\}$ to indicate the choice over hours worked, with $0$ representing no hours worked, $1 representing part-time and $2$ representing full-time employment. We are therefore going to consider a static model with 9 discrete choices. Note that there is a Type 1 extreme value shock iid over time and across individuals, $\epsilon_j$ for choice $j$. The shocks are scaled by $\sigma$.

Utility for individual $i$ at time $t$ is given by:
$$
U_{itj} = \log(\max\{50, Y_{it}(W_{it}H_j)\}) + \alpha_l\log(112-H_j)-\alpha_{P,1}1\{P_j>0\} - \alpha_{P,2}1\{P_j>1\}
$$
where $Y_{it}$ is net income for individual $i$ at time $t$ and depends on their earnings $(W_{it}H_j)$, the number of children in the household, and the policy environment in the individual's state of residenceeeee. 


Wages are a deterministic function of age only:
$$
\log(W_{it}) = \gamma_0 + \gamma_1Age_{it}+ \gamma_2Age_{it}^2
$$
and are measured with normally distributed iid measurement error:
$$
\log(W_{it}^0) = \log(W_{it}) + \xi_{it}, \quad  \xi_{it} \sim N(0, \sigma_W^2)
$$

Next, we calculate utility and index choices.

In [29]:
j_idx(p,h) = p*3 + h + 1
function utility(p,h,soi,cpi,year,num_kids,wage,pars::NamedTuple)
    (;αl,Hgrid,σ,αP) = pars
    hrs = Hgrid[1+h]
    earn = wage * hrs
    net_income = max(50.,Transfers.budget(earn,0.,soi,earn,num_kids,cpi,p))
    return log(net_income) + αl*log(112-hrs)-αP[1]*(p>0)-αP[2]*(p>1)
end

pars = (;αl = 1., σ=1.,σW=1.,γ= zeros(3),
        Hgrid = [0.,20.,40.], αP = [0.,0.])

(αl = 1.0, σ = 1.0, σW = 1.0, γ = [0.0, 0.0, 0.0], Hgrid = [0.0, 20.0, 40.0], αP = [0.0, 0.0])

Choice probabilities are given by the logit formula.

In [25]:
function choice_prob!(logP,it,data,pars) # note that logP is modified in place by the ! in the function name
    (;σ,γ) = pars
    wage = exp(γ[1]+γ[2]*data.age[it]+γ[3]*data.age[it]^2)
    denom = 0.
    umax = -Inf
    for p in 0:2
        for h in 0:2
            j = j_idx(p,h)
            u = utility(p,h,data.soi[it],data.cpi[it],data.year[it],data.num_kids[it],wage,data,pars)
            LogP[j] = u/σ
            umax = u>umax ? u : umax # if u>umax, umax = u, else umax = umax
        end
    end
    logP[:] .-= umax/σ # elementwise subtraction
    denom = log(sum(exp.(logP))) # log of sum of exponentials
    logP[:] .-= denom # normalise choice probabilities
end

choice_prob! (generic function with 1 method)

Now define the log-likelihood.

In [28]:
function log_likelihood(logP,it,data,pars)
    (;γ,σW) = pars
    logL = 0.
    if !data.wage_missing[it]
        wage = -((data.logwage[it]-γ[1]-γ[2]*data.age[it]-γ[3]*data.age[it]^2)/σW)^2 - log(σW)
    end
    choice_prob!(logP,it,data,pars)
    j = j_idx(data.P[it],data.H[it])
    logL += logP[j]
    return logL
end

function update(x,p)
    αl = exp(x[1])
    σ = exp(x[2])
    γ = x[3:5] # slice x from 3 to 5
    σW = exp(x[6])
    αP = x[7:8]
    return (;p...,αl,σ,σW,αP,γ)
end

function log_likelihood(x,data,pars)
    pars = update(x,pars)
    logP = zeros(eltype(x),9)
    logL = 0.
    for it in eachindex(data.P)
        logL += log_likelihood(logP,it,data,pars)
    end
    return LogL
end


log_likelihood (generic function with 2 methods)

Finally, use Optim to maximise that log-likelihood:

In [31]:
using ForwardDiff

ArgumentError: ArgumentError: Package ForwardDiff not found in current path.
- Run `import Pkg; Pkg.add("ForwardDiff")` to install the ForwardDiff package.

In [30]:
x0 = zeros(8)
x0 = [0.,0.,log(10.),0.,0.,0.,0.,0.]
res = optimize(x->-log_likelihood(x,data_mle,pars),x0,BFGS(),autodiff=:forward,Optim.Options(show_trace=true))
pars = update(res.minimizer,pars)

MethodError: MethodError: no method matching utility(::Int64, ::Int64, ::Int64, ::Float64, ::Int64, ::Int64, ::ForwardDiff.Dual{ForwardDiff.Tag{var"#91#92", Float64}, Float64, 8}, ::@NamedTuple{P::Vector{Int64}, H::Vector{Int64}, year::Vector{Int64}, age::Vector{Int64}, soi::Vector{Int64}, num_kids::Vector{Int64}, cpi::Vector{Float64}, logwage::Vector{Float64}, wage_missing::BitVector}, ::@NamedTuple{αl::ForwardDiff.Dual{ForwardDiff.Tag{var"#91#92", Float64}, Float64, 8}, σ::ForwardDiff.Dual{ForwardDiff.Tag{var"#91#92", Float64}, Float64, 8}, σW::ForwardDiff.Dual{ForwardDiff.Tag{var"#91#92", Float64}, Float64, 8}, γ::Vector{ForwardDiff.Dual{ForwardDiff.Tag{var"#91#92", Float64}, Float64, 8}}, Hgrid::Vector{Float64}, αP::Vector{ForwardDiff.Dual{ForwardDiff.Tag{var"#91#92", Float64}, Float64, 8}}})

Closest candidates are:
  utility(::Any, ::Any, ::Any, ::Any, ::Any, ::Any, ::Any, ::NamedTuple)
   @ Main c:\Users\bayle\Documents\Github\micro_labour\hw2\hw2.ipynb:2
  utility(::Any, ::Any,

#### **Question 2**

#### **Question 3**

#### **Question 4**